# 因篇幅原因，对卷积神经网络的讲解详细见我的博客。
<br>

# <a href="https://blog.csdn.net/weixin_43217928/article/details/88426172" target="_blank">【一文读懂卷积神经网络（一）】可能是你看过的最全的CNN（步长为1，无填充）</a>
<br>

# <a href="https://blog.csdn.net/weixin_43217928/article/details/88555389" target="_blank">【一文读懂卷积神经网络（二）】可能是你看过的最全的CNN（步长为1，有填充）</a>
<br>

# <a href="https://blog.csdn.net/weixin_43217928/article/details/88564517" target="_blank">【一文读懂卷积神经网络（三）】可能是你看过的最全的CNN（步长不为1，无填充）</a>
<br>

# <a href="https://blog.csdn.net/weixin_43217928/article/details/88597216" target="_blank">【一文读懂卷积神经网络（四）】可能是你看过的最全的CNN（步长不为1，有填充）</a>
<br>

In [1]:
import numpy as np
import h5py
import random

In [2]:
class cnnModel():
    def __init__(self,conv_layers,fc_layers,Xtrain,ytrain,
                 batch_size=16):
        self.conv_layers=conv_layers  #卷积层结构
        self.fc_layers=fc_layers      #全连接层结构
        self.convWeights = [np.random.randn(layer['filterSize'], layer['filterSize'], layer['filterChannels'],layer['filterNums']) for layer in self.conv_layers]
        self.convBiases = [np.random.randn(1, layer['filterNums']) for layer in self.conv_layers]
        self.fcWeights=[]
        self.Xtrain=Xtrain
        self.ytrain=ytrain
        self.batch_size=batch_size
        self.batch_index=0
        self.data_index=[i for i in range(Xtrain.shape[0])]
        self.caches={}
        print("initial conv params complete!")



    def initFCParams(self,flattenNums):
        #fcWeights非空，说明已经初始化过了
        if self.fcWeights:
            return

        self.fc_layers.insert(0,flattenNums)
        self.fcWeights=[np.random.randn(self.fc_layers[dim],self.fc_layers[dim+1])*np.sqrt(2 / self.fc_layers[dim]) for dim in range(len(self.fc_layers)-1)]
        self.fcBiases=[np.zeros((1,self.fc_layers[dim])) for dim in range(1,len(self.fc_layers))]
        print("initial FC params complete!")



    def zero_pad(self,X,pad_size):   #对样本进行填充
        #pad_size为tuple
        # 填充顺序依次为样本数，高度，宽度，通道数
        X_paded=np.pad(X,((0,0),pad_size,pad_size,(0,0)),'constant')
        return X_paded


    def delete_pad(self,dA,pad_size):
        pad_h_size,pad_w_size=pad_size  #填充的大小
        m,n_H_prev,n_W_prev,n_C=dA.shape  #填充后的大小
        #计算出填充前的大小
        n_H=n_H_prev-2*pad_h_size
        n_W=n_W_prev-2*pad_w_size
        #需要删除的行和列的list
        delete_h_list=[h for h in range(pad_h_size)]
        delete_w_list=[w for w in range(pad_w_size)]
        for i in range(pad_h_size):
            delete_h_list.append((i+n_H+pad_h_size))
        for j in range(pad_w_size):
            delete_w_list.append((j+n_W+pad_w_size))

        dA=np.delete(dA,delete_h_list,axis=1)
        dA=np.delete(dA,delete_w_list,axis=2)

        return dA



    def zero_insert(self,dZ,insert_size):
        m,H,W,C=dZ.shape
        dZout=dZ.copy()
        insert_values_h=np.zeros((insert_size,W))
        insert_values_w=np.zeros((H,insert_size))
        for h in range(H-1):
            dZout=np.insert(dZout,h+1,values=insert_values_h,
                            axis=1)
        for w in range(W-1):
            dZout=np.insert(dZout,w+1,values=insert_values_w,
                            axis=2)
        return dZout



    def relu(self,Z):
        return np.maximum(0,Z)



    def relu_back(self,dA,Z):
        dZ=dA.copy()
        dZ[Z<=0]=0
        return dZ



    def softmax(self,Z):
        return np.divide(np.exp(Z), np.sum(np.exp(Z), axis=1).reshape(-1,1))


    def softmax_back(self,A,y):
        return A-y


    def max_pool_forward(self,Ain,poolSize,strides=1,
                         mode='SAME'):
        #Ain输入数据
        #dim为当前层数
        #poolSize为池化核大小
        #strides为池化的步长
        #mode为是否填充
        batch_size, n_H_prev, n_W_prev, n_C_prev = Ain.shape
        # batch_size批量输入数据的个数，n_H_prev输入数据的高，n_W_prev输入数据的宽，n_C_prev输入数据的通道数
        # self.caches['unpadAinSize'+str(dim+1)]=Ain.shape
        padSize=()
        if mode=='SAME':  #如果需要填充的话
            pad_h_size = int((strides * (n_H_prev - 1) - n_H_prev + poolSize) / 2)  # 在高度上填充的大小
            pad_w_size = int((strides * (n_W_prev - 1) - n_W_prev + poolSize) / 2)  # 在宽度上填充的大小
            padSize=(pad_h_size,pad_w_size)
            # self.caches['padSize' + str(dim + 1)] = (pad_h_size,pad_w_size)
            Ain = self.zero_pad(Ain, pad_size=(pad_h_size, pad_w_size))  # 进行0填充

            n_H = n_H_prev  # 卷积后图像的高
            n_W = n_W_prev  # 卷积后图像的宽
        elif mode=='VALID':
            n_H = int(np.floor((n_H_prev - poolSize) / strides) + 1)
            # 卷积后图像的高
            n_W = int(np.floor((n_W_prev - poolSize) / strides) + 1)
            # 卷积后图像的宽
        else:
            raise Exception("Invalid padding method!")
        # self.caches['poolAin_pad' + str(dim + 1)] = Ain
        Aout=np.zeros((batch_size,n_H,n_W,n_C_prev))
        for i in range(batch_size):
            for h in range(n_H):
                for w in range(n_W):
                    for c in range(n_C_prev):
                        h_start=h*strides
                        h_end=h_start+poolSize
                        w_start=w*strides
                        w_end=w_start+poolSize
                        Aout[i,h,w,c]=np.max(Ain[i,
h_start:h_end,w_start:w_end,c])
        #返回的Aout是池化后的结果，如果mode=='SAME',则Ain是填充后的Ain，否则为未填充的Ain，padSize为填充的大小，若未填充则为空
        return Aout,Ain,padSize



    def average_pool_forward(self,Ain,poolSize,strides=1,
                             mode='SAME'):
        batch_size, n_H_prev, n_W_prev, n_C_prev = Ain.shape
        # batch_size批量输入数据的个数，n_H_prev输入数据的高，n_W_prev输入数据的宽，n_C_prev输入数据的通道数
        # self.caches['unpadAinSize' + str(dim + 1)] = Ain.shape
        padSize=()
        if mode == 'SAME':
            pad_h_size = int((strides * (n_H_prev - 1) - n_H_prev + poolSize) / 2)  # 在高度上填充的大小
            pad_w_size = int((strides * (n_W_prev - 1) - n_W_prev + poolSize) / 2)  # 在宽度上填充的大小
            # self.caches['padSize'+str(dim+1)]=(pad_h_size,pad_w_size)
            padSize=(pad_h_size,pad_w_size)
            Ain = self.zero_pad(Ain, pad_size=(pad_h_size, pad_w_size))  # 进行0填充
            n_H = n_H_prev  # 卷积后图像的高
            n_W = n_W_prev  # 卷积后图像的宽
        elif mode == 'VALID':
            n_H = int(np.floor((n_H_prev - poolSize) / strides) + 1)
            # 卷积后图像的高
            n_W = int(np.floor((n_W_prev - poolSize) / strides) + 1)
            # 卷积后图像的宽
        else:
            raise Exception("Invalid padding method!")
        # self.caches['poolAin_pad' + str(dim + 1)] = Ain
        Aout = np.zeros((batch_size, n_H, n_W, n_C_prev))
        for i in range(batch_size):
            for h in range(n_H):
                for w in range(n_W):
                    for c in range(n_C_prev):
                        h_start = h * strides
                        h_end = h_start + poolSize
                        w_start = w * strides
                        w_end = w_start + poolSize
                        Aout[i, h, w, c] = np.mean(Ain[i, h_start:h_end, w_start:w_end, c])
        # 返回的Aout是池化后的结果，如果mode=='SAME',则Ain是填充后的Ain，否则为未填充的Ain，padSize为填充的大小，若未填充则为空
        return Aout,Ain,padSize



    def fc_forward(self,A,predict):    #全连接层前向传播
        if not predict:
            self.caches['FCA0']=A
        for dim in range(1,len(self.fc_layers)):
            Z=A.dot(self.fcWeights[dim-1])+self.fcBiases[dim-1]
            if dim==(len(self.fc_layers)-1):  #如果是最后一层
                A=self.softmax(Z)
            else:
                A=self.relu(Z)
            if not predict:
                self.caches['FCA'+str(dim)]=A
                self.caches['FCZ'+str(dim)]=Z
        return A



    def conv_forward(self,A,convFilter,bias,padding='SAME',
                     strides=1):   #卷积层前向传播
        batch_size, n_H_prev, n_W_prev, n_C_prev = A.shape
        # batch_size批量输入数据的个数，n_H_prev输入数据的高，n_W_prev输入数据的宽，n_C_prev输入数据的通道数
        filterSize, filterSize, filterChannels, filterNums = convFilter.shape
        # filterSize卷积核大小，filterChannels卷积核的通道数，应该和n_C_prev相等，filterNums卷积核个数
        padSize=()
        if padding=='SAME':
            pad_h_size=int((strides*(n_H_prev-1)-n_H_prev+filterSize)/2 ) #在高度上填充的大小
            pad_w_size = int((strides * (n_W_prev - 1) - n_W_prev + filterSize) / 2 )  #在宽度上填充的大小
            padSize=(pad_h_size,pad_w_size)
            A=self.zero_pad(A,pad_size=(pad_h_size,pad_w_size))      #进行0填充
            n_H=n_H_prev   #卷积后图像的高
            n_W=n_W_prev   #卷积后图像的宽
        elif padding=='VALID':
            n_H=int(np.floor((n_H_prev-filterSize)/strides)+1)
            # 卷积后图像的高
            n_W=int(np.floor((n_W_prev-filterSize)/strides)+1)
            # 卷积后图像的宽
        else:
            raise Exception("Invalid padding method!")
        #batch_size为一个batch的样本数，n_H为卷积后的高，n_W为卷积后的宽，filterNums为卷积后的通道数
        Z=np.zeros((batch_size,n_H,n_W,filterNums))
        for c in range(filterNums):
            for i in range(batch_size):
                for h in range(n_H):
                    for w in range(n_W):
                        h_start=h*strides
                        h_end=h_start+filterSize
                        w_start=w*strides
                        w_end=w_start+filterSize
                        Z[i,h,w,c]=np.sum(
np.multiply(A[i,h_start:h_end,w_start:w_end,:],convFilter[:,:,:,c]))+bias[0,c]
        #返回卷积后的Z，若padding='SAME'，则A为填充后的A，否则就为传入的A
        return Z,A,padSize



    def forward(self,xs,predict=False):      #模型总的前向传播，由卷积层前向传播与前连接层前向传播组成
        Aout=xs

        for dim,layer in enumerate(self.conv_layers):
            #Aout为上一层池化的输出，如果为第一层则是输入样本
            Z,Apad,padSize=self.conv_forward(Aout,
self.convWeights[dim],self.convBiases[dim],layer['conv_padding'],layer['strides'])
            #Z为卷积后的结果，若layer['conv_padding']=='SAME'，则Apaded为填充后的Aout，若layer['conv_padding']=='VALID'，则Apaded就是Aout

            #激活函数
            Ain=self.relu(Z)


            if layer['poolType']=='MAX':
                Aout,Ain_pad,padSize=self.max_pool_forward(Ain,
layer['poolSize'],layer['poolSize'],layer['pool_padding'])
            else:
                Aout,Ain_pad,padSize=self.average_pool_forward(Ain, layer['poolSize'], layer['poolSize'], layer['pool_padding'])
            #如果当前不是预测，就将反向传播时需要用的保存起来
            if not predict:
                self.caches['convPadSize' + str(dim)] = padSize
                # 将当前的Apaded储存起来供反向传播时使用
                self.caches['convAout' + str(dim)] = Apad
                # 将Z保存起来，供反向传播时使用
                self.caches['convZ' + str(dim + 1)] = Z
                self.caches['poolAin'+str(dim+1)]=Ain_pad
                self.caches['poolPadSize'+str(dim+1)]=padSize
                self.caches['poolAout'+str(dim+1)]=Aout

        #全连接层
        A=Aout.reshape((Aout.shape[0],-1))
        
        if not predict:
            self.initFCParams(A.shape[1])

        AL=self.fc_forward(A,predict)

        return AL


    def fc_backward(self,AL,y):   #全连接层的后向传播
        grads={}
        L = len(self.fc_layers)
        m = AL.shape[0]
        dZ = self.softmax_back(AL,y)
        grads['FCdW' + str(L - 2)] = (1. / m) * self.caches['FCA' + str(L - 2)].T.dot(dZ)
        grads['FCdb' + str(L - 2)] = (1. / m) * np.sum(dZ, axis=0, keepdims=True)
        dA = dZ.dot(self.fcWeights[L-2].T)
        for dim in reversed(range(1, L-1)):
            dZ=self.relu_back(dA,self.caches['FCZ'+str(dim)])
            grads['FCdW' + str(dim - 1)] = (1. / m) * self.caches['FCA' + str(dim - 1)].T.dot(dZ)
            grads['FCdb' + str(dim - 1)] = (1. / m) * np.sum(dZ, axis=0, keepdims=True)
            dA = dZ.dot(self.fcWeights[dim-1].T)
        return grads,dA



    def conv_backward(self,dZ,convFilter,Aout,filterSize,
                      strides):   #卷积层的后向传播
        #对卷积核进行顺时针翻转180度
        f_i,f_j,f_k,f_l=convFilter.shape
        for k in range(f_k):
            for l in range(f_l):
                convFilter[:,:,k,l]=np.rot90(convFilter[:,:,k,l],
k=-1)
                convFilter[:,:,k,l]=np.rot90(convFilter[:,:,k,l],
k=-1)


        batch_size,n_H_prev,n_W_prev,n_C_prev=dZ.shape
        batch_size,n_H,n_W,n_C=Aout.shape
        dW = np.zeros((filterSize, filterSize, n_C, n_C_prev))
        for num in range(n_C_prev):  
            for c in range(n_C):
                for m in range(filterSize):
                    for n in range(filterSize):
                        for i in range(n_H_prev):
                            for j in range(n_W_prev):
                                dW[m, n, c, num] += np.sum(np.multiply(Aout[:, i * strides + m, j * strides + n, c], dZ[:, i, j, num]))
                                
        dW = dW / batch_size
        db = np.sum(np.sum(np.sum(dZ, axis=0), axis=0), axis=0, keepdims=True) / batch_size

        dA=np.zeros_like(Aout)
        if strides>1:
            dZ_inserted = self.zero_insert(dZ, strides - 1)
            batch_size, n_H_prev, n_W_prev, n_C_prev=dZ_inserted.shape
        else:
            dZ_inserted=dZ
        pad_h_size = int((n_H - 1 - n_H_prev + filterSize) / 2)
        pad_w_size = int((n_W - 1 - n_W_prev + filterSize) / 2)
        dZ_inserted_paded=self.zero_pad(dZ_inserted,pad_size=(pad_h_size,pad_w_size))
        for c in range(n_C):  
            for i in range(batch_size):  
                for h in range(n_H):  
                    for w in range(n_W):  
                        dA[i,h,w,c]=np.sum(
np.multiply(dZ_inserted_paded[i,h:h+filterSize,w:w+filterSize,:],convFilter[:,:,c,:]))

        return dA,dW,db




    def max_pool_backward(self,dAout,Ain,filterSize,strides):
        #正向传播时的Ain，如果正向传播时做了填充，这就是填充后的Ain，如果没有，就是未填充的Ain
        dAin=np.zeros_like(Ain)
        m,n_H,n_W,n_C=dAout.shape
        for i in range(m):
            for h in range(n_H):
                for w in range(n_W):
                    for c in range(n_C):
                        h_start=h*strides
                        h_end=h_start+filterSize
                        w_start=w*strides
                        w_end=w_start+filterSize
                        mask=(Ain[i,h_start:h_end,
w_start:w_end,c]==np.max(Ain[i,h_start:h_end,w_start:w_end,c]))
                        dAin[i,h_start:h_end,
w_start:w_end,c]=mask*dAout[i,h,w,c]
        return dAin



    def average_pool_backward(self,dAout,Ain,filterSize,strides):
        # 正向传播时的Ain，如果正向传播时做了填充，这就是填充后的Ain，如果没有，就是未填充的Ain
        dAin = np.zeros_like(Ain)
        m, n_H, n_W, n_C = dAout.shape
        for i in range(m):
            for h in range(n_H):
                for w in range(n_W):
                    for c in range(n_C):
                        h_start = h * strides
                        h_end = h_start + filterSize
                        w_start = w * strides
                        w_end = w_start + filterSize
                        dAin[i,h_start:h_end,
w_start:w_end,c]+=np.ones((filterSize,filterSize))*(dAout[i,h,w,c]/(filterSize**2))

        return dAin



    def backward(self,AL,y):      #模型总得后向传播，由全连接层的后向传播与卷积层的后向传播组成
        grads,dA=self.fc_backward(AL,y)
        L=len(self.conv_layers)
        dAout=dA.reshape(self.caches['poolAout'+str(L)].shape)
        for dim in reversed(range(L)):
            layer=self.conv_layers[dim]
            
            if layer['poolType']=='MAX':
                dAin=self.max_pool_backward(dAout,
self.caches['poolAin'+str(dim+1)],layer['poolSize'],layer['poolSize'])
            else:
                dAin = self.average_pool_backward(dAout, self.caches['poolAin' + str(dim + 1)], layer['poolSize'],
layer['poolSize'])
                
            if layer['pool_padding']=='SAME':
                dAin=self.delete_pad(dAin,
pad_size=self.caches['poolPadSize' + str(dim + 1)])
                
            dZ=self.relu_back(dAin,self.caches['convZ'+str(dim+1)])
            dAout,dW,db=self.conv_backward(dZ,
self.convWeights[dim],self.caches['convAout'+str(dim)],layer['filterSize'],layer['strides'])
            
            if layer['conv_padding']=='SAME':
                dAout=self.delete_pad(dAout,
self.caches['convPadSize'+str(dim)])

            grads['CONVdW'+str(dim)]=dW
            grads['CONVdb'+str(dim)]=db

        return grads




    def updateParams(self,grads):   #更新参数，在后向传播结束时进行
        for i in range(len(self.conv_layers)):
            self.convWeights[i]=self.convWeights[i]-self.learning_rate*grads['CONVdW'+str(i)]
            self.convBiases[i]=self.convBiases[i]-self.learning_rate*grads['CONVdb'+str(i)]

        for j in range(len(self.fc_layers)-1):
            self.fcWeights[j]=self.fcWeights[j]-self.learning_rate*grads['FCdW'+str(j)]
            self.fcBiases[j]=self.fcBiases[j]-self.learning_rate*grads['FCdb'+str(j)]


    def next_batch(self):
        m=self.Xtrain.shape[0]
        start = min(self.batch_index, m)
        if start==m:
            self.batch_index=0
            start=self.batch_index
        if self.batch_index==0:  #一轮数据遍历完了，重新打乱数据
            random.shuffle(self.data_index)
        end=min(start+self.batch_size,m)
        xs=self.Xtrain[start:end]
        ys=self.ytrain[start:end]
        self.batch_index=end
        return xs,ys


    def GredientDescentOptimizer(self,learning_rate,
                                 training_steps=200):
        self.learning_rate=learning_rate
        xs,ys=self.next_batch()
        for step in range(training_steps):
            AL=self.forward(xs)
            grads=self.backward(AL,ys)
            self.updateParams(grads)
            if step%5==0:
                accuracy=self.calcAccuracy(
self.Xtrain[:100,:,:,:],self.ytrain[:100,:])
                loss=self.calcLoss(
self.Xtrain[:100,:,:,:],self.ytrain[:100,:])
                print("after %d training steps,the accuracy on test dataset is :%g%%,the loss is :%g"%(step,accuracy,loss))
        print("training complete!")




    def calcAccuracy(self,X,y):   #给定输入数据和标签，计算当前模型下的正确率
        h=np.argmax(self.forward(X,predict=True),1).reshape(-1,1)
        y=np.argmax(y,1).reshape(-1,1)
        accuracy=np.sum(h==y)/len(h)*100
        return accuracy


    def calcLoss(self,X,y):    #给定输入数据和标签，计算当前模型下的损失

        h=self.forward(X,predict=True)
        h=np.log(h)
        loss=np.sum(np.multiply(h,-y))/y.shape[0]
        return loss

In [3]:
#加载数据的function
def load_dataset():
    train_dataset = h5py.File('datasets/train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:])  # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:])  # your train set labels

    test_dataset = h5py.File('datasets/test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:])  # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:])  # your test set labels

    classes = np.array(test_dataset["list_classes"][:])  # the list of classes

    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [4]:
#加载数据
train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes=load_dataset()

trainX=train_set_x_orig/255  #shape(1080,64,64,3)样本数，高，宽，通道数
testX=test_set_x_orig/255
trainy=train_set_y_orig.T
testy=test_set_y_orig.T
#转为one-hot编码
n_class = testy.max() + 1
trainy=np.eye(n_class)[trainy].reshape(-1,6)

In [5]:
print(trainX.shape)
print(trainy.shape)
print(testX.shape)
print(testy.shape)

(1080, 64, 64, 3)
(1080, 6)
(120, 64, 64, 3)
(120, 1)


In [6]:
#构造网络层结构的function
def generateNNLayerStruct():
    conv_layers=[]
    cnnLayerNums=int(input("请输入卷积层的层数(please input your convolution layer's nums)："))
    print("您为卷积层设置了%d层(You set %d layers for convolution)"%(cnnLayerNums,cnnLayerNums))
    for layer in range(int(cnnLayerNums)):
        dict={}
        print("\n")
        print("请输入第%d层的参数(please input %d layer's params):"%(layer+1,layer+1))
        print("\n")
        #卷积层参数
        dict['filterSize']=int(input("请输入当前层卷积核的大小(please input current layer's convolution filter size):"))
        dict['filterChannels']=int(input("请输入当前层卷积核的通道数，必须和输入数据的通道数相同(please input current layer's convolution filter channels,it must be equal to input data's channels):"))
        dict['filterNums']=int(input("请输入当前层卷积核的数目(please input current layer's convolution filter nums):"))
        dict['conv_padding']=str(input("请输入当前层卷积层的填充模式，分为SAME（填充）和VALID（不填充）两种(please input current layer's convolution layer's pad mode-SAME(pad) or VALID(non-pad)):"))
        dict['strides']=int(input("请输入当前层卷积层的步长(please input current layer's convolution layer's strides):"))
        #池化层参数
        dict['poolSize']=int(input("请输入当前层池化核的大小(please input current layer's pooling filter size):"))
        dict['poolType']=str(input("请输入当前层池化的模式，MAX（最大池化）和AVERAGE（均值池化）两种(please input current layer's pooling layer's mode-MAX(max pooling) or AVERAGE(average pooling)):"))
        dict['pool_padding']=str(input("请输入当前层池化的填充模式，分为SAME（填充）和VALID（不填充）两种(please input current layer's pooling layer's pad mode-SAME(pad) or VALID(non-pad)):"))
        conv_layers.append(dict)
    
    print("\n卷积层参数输入完毕!（input convolution layer's params complete!）\n")
    
    fc_layers=[]
    fcLayerNums=int(input("请输入全连接层的层数，只用计算输入隐藏层和输出层，不用输入层(please input your full connect layer's nums,only count hidden layers and output layers)："))
    print("您为全连接层设置了%d层(You set %d layers for full connect)"%(fcLayerNums,fcLayerNums))
    for layer in range(int(fcLayerNums)):
        print("\n")
        print("请输入第%d层的节点数(please input %d layer's node nums):"%(layer+1,layer+1))
        print("\n")
        nums=input("请输入当前层的节点数(please input current layer's node nums)：")
        fc_layers.append(int(nums))
    
    print("\n全连接层参数输入完毕!（input full connect layer's params complete!）\n")
    
    return conv_layers,fc_layers
    
 

In [13]:

# conv_layers=[{'filterSize':5,'filterChannels':3,'filterNums':8,'conv_padding':'SAME','strides':1,'poolSize':8,'poolType':'MAX','pool_padding':'SAME'},{'filterSize':3,'filterChannels':8,'filterNums':16,'conv_padding':'SAME','strides':1,'poolSize':4,'poolType':'MAX','pool_padding':'SAME'}]
# fc_layers=[6]

#调用generateNNLayerStruct来设定网络层参数
conv_layers,fc_layers=generateNNLayerStruct()

请输入卷积层的层数(please input your convolution layer's nums)：2
您为卷积层设置了2层(You set 2 layers for convolution)


请输入第1层的参数(please input 1 layer's params):


请输入当前层卷积核的大小(please input current layer's convolution filter size):5
请输入当前层卷积核的通道数，必须和输入数据的通道数相同(please input current layer's convolution filter channels,it must be equal to input data's channels):3
请输入当前层卷积核的数目(please input current layer's convolution filter nums):8
请输入当前层卷积层的填充模式，分为SAME（填充）和VALID（不填充）两种(please input current layer's convolution layer's pad mode-SAME(pad) or VALID(non-pad)):SAME
请输入当前层卷积层的步长(please input current layer's convolution layer's strides):1
请输入当前层池化核的大小(please input current layer's pooling filter size):8
请输入当前层池化的模式，MAX（最大池化）和AVERAGE（均值池化）两种(please input current layer's pooling layer's mode-MAX(max pooling) or AVERAGE(average pooling)):MAX
请输入当前层池化的填充模式，分为SAME（填充）和VALID（不填充）两种(please input current layer's pooling layer's pad mode-SAME(pad) or VALID(non-pad)):SAME


请输入第2层的参数(please input 2 layer's params):


请输入当前层

In [11]:
myCNN=cnnModel(conv_layers,fc_layers,trainX,trainy)
myCNN.GredientDescentOptimizer(0.001)

# 因为自己实现的代码有大量的for循环,且没有gpu加速，就不在这跑了，本人做了个实验，大小为100的一个batch大约要跑接近1个小时左右，完完全全被tensorflow-gpu完爆了，该代码只是用于学习加深了解卷积神经网络。

In [9]:
import tensorflow as tf
import random

batch_size=100
IMAGE_SIZE=64
NUM_CHANNELS=3
OUTPUT_NODE=6
BATCH_INDEX=0



def getBatch(indexs,batch_size):
    global BATCH_INDEX
    if BATCH_INDEX==len(indexs):
        BATCH_INDEX=0
        random.shuffle(indexs)
    end=min(BATCH_INDEX+batch_size,len(indexs))
    batchIndexs=indexs[BATCH_INDEX:end]
    BATCH_INDEX=end
    
    return batchIndexs,indexs




def forward(conv_layers,fc_layers,X,firstFC):
    count = 1
    pool=X
    for dim,layer in enumerate(conv_layers):
        # 卷积层
        with tf.variable_scope(('layer'+str(count)+'-conv'+str(dim+1)),reuse=tf.AUTO_REUSE):
            conv_weights=tf.get_variable("weights",[layer['filterSize'],layer['filterSize'],layer['filterChannels'],layer['filterNums']],initializer=tf.truncated_normal_initializer(stddev=0.1))
            conv_biases=tf.get_variable("biases",layer['filterNums'],initializer=tf.constant_initializer(0.0))
            conv=tf.nn.conv2d(pool,conv_weights,[1,layer['strides'],layer['strides'],1],padding=layer['conv_padding'])
            relu=tf.nn.relu(tf.nn.bias_add(conv,conv_biases))
            count+=1

        #池化层
        with tf.variable_scope(('layer'+str(count)+'-pool'+str(dim+1)),reuse=tf.AUTO_REUSE):
            if layer['poolType']=='MAX':
                pool=tf.nn.max_pool(relu,[1,layer['poolSize'],layer['poolSize'],1],[1,layer['poolSize'],layer['poolSize'],1],layer['pool_padding'])
            else:
                pool = tf.nn.avg_pool(relu, [1, layer['poolSize'], layer['poolSize'], 1], [1,layer['poolSize'],layer['poolSize'],1],layer['pool_padding'])
            count+=1


    pool_shape=pool.get_shape().as_list()
    #展开
    nodes=pool_shape[1]*pool_shape[2]*pool_shape[3]
    flatten=tf.reshape(pool,[pool_shape[0],nodes])
    FCLAYERS=fc_layers.copy()
    if firstFC:
        FCLAYERS.insert(0,nodes)
        firstFC==False
    for dim in range(len(FCLAYERS)-1):
        with tf.variable_scope(('layer'+str(count)+'-fc'+str(dim+1)),reuse=tf.AUTO_REUSE):
            fc_weights=tf.get_variable("weights",[FCLAYERS[dim],FCLAYERS[dim+1]],initializer=tf.truncated_normal_initializer(stddev=0.1))
            fc_biases=tf.get_variable("biases",
FCLAYERS[dim+1],initializer=tf.constant_initializer(0.0))

            flatten=tf.matmul(flatten,fc_weights)+fc_biases
            if dim!=(len(FCLAYERS)-2):
                flatten=tf.nn.relu(flatten)
            count+=1

    logit=flatten
    return logit



def train(conv_layers,fc_layers,Xtrain,ytrain,learning_rate,
          TRAINING_STEPS=1000):
    X=tf.placeholder(tf.float32,[batch_size,IMAGE_SIZE,IMAGE_SIZE,NUM_CHANNELS],name="Xinput")
    y=tf.placeholder(tf.float32,[None,OUTPUT_NODE],name="yinput")
    h=forward(conv_layers,fc_layers,X,True)
    cross_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h,labels=tf.argmax(y,1))
    loss=tf.reduce_mean(cross_entropy)
    train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        indexs=[i for i in range(X.shape[0])]
        for step in range(TRAINING_STEPS):
            batchIndexs,indexs=getBatch(indexs,batch_size)
            _,loss_value=sess.run([train_step,loss],feed_dict={X:Xtrain[batchIndexs,:,:,:],y:ytrain[batchIndexs,:]})
            if step%100==0:
                print("after %d training steps, loss on training batch is %g" % (step, loss_value))

In [10]:
train(conv_layers,fc_layers,trainX,trainy,0.1)

after 0 training steps, loss on training batch is 1.79522
after 100 training steps, loss on training batch is 1.73289
after 200 training steps, loss on training batch is 1.43567
after 300 training steps, loss on training batch is 0.871796
after 400 training steps, loss on training batch is 0.365442
after 500 training steps, loss on training batch is 0.0639015
after 600 training steps, loss on training batch is 0.0224951
after 700 training steps, loss on training batch is 0.0126631
after 800 training steps, loss on training batch is 0.00854564
after 900 training steps, loss on training batch is 0.00633865
